In [12]:
!pip install gensim nltk contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 13.3 MB/s eta 0:00:00


In [13]:
import re
import numpy as np
import pandas as pd
import nltk
import contractions
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import gensim.downloader as api

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [ ]:
w2v_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def tweet_to_vector(tokens, model, vector_size=300):
    vectors = [model[word] for word in tokens if word in model]
    if len(vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(vectors, axis=0)

In [ ]:
df = pd.read_csv("Tweets.csv")
df = df[['airline_sentiment', 'text']]

df['tokens'] = df['text'].apply(preprocess_text)
X = np.vstack(df['tokens'].apply(lambda x: tweet_to_vector(x, w2v_model)))
y = df['airline_sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", accuracy)

Test Accuracy: 0.7742486338797814


In [ ]:
def predict_tweet_sentiment(model, w2v_model, tweet):
    tokens = preprocess_text(tweet)
    vector = tweet_to_vector(tokens, w2v_model).reshape(1, -1)
    return model.predict(vector)[0]

In [ ]:
predict_tweet_sentiment(
    model,
    w2v_model,
    "The flight was delayed and staff were rude"
)

'negative'

Problem 2: Hugging Face BERT Pipeline

(IMDb Sentiment Analysis)

In this task, we fine-tune a pre-trained BERT model for binary sentiment classification using the IMDb dataset.

Pipeline Components:

Hugging Face Datasets

Tokenization using BERT tokenizer

Model fine-tuning

Evaluation (Accuracy & F1-score)

Model saving & inference

In [2]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

In [4]:
dataset = load_dataset("imdb")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Text is tokenized using bert-base-uncased tokenizer with:

Padding

Truncation

Fixed max length

This prepares the text for transformer input.

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We evaluate performance using:

Accuracy → overall correctness

F1-score → balance between precision & recall

In [6]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }

In [8]:
training_args = TrainingArguments(
    output_dir="./bert-imdb",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs"
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-3025579641.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.277300,0.251057,0.914680,0.911965
2,0.160200,0.337968,0.921280,0.921979


TrainOutput(global_step=6250, training_loss=0.2479502862548828, metrics={'train_runtime': 3417.6182, 'train_samples_per_second': 14.63, 'train_steps_per_second': 1.829, 'total_flos': 6577776384000000.0, 'train_loss': 0.2479502862548828, 'epoch': 2.0})

In [10]:
trainer.save_model("bert-imdb-sentiment")
tokenizer.save_pretrained("bert-imdb-sentiment")

('bert-imdb-sentiment/tokenizer_config.json',
 'bert-imdb-sentiment/special_tokens_map.json',
 'bert-imdb-sentiment/vocab.txt',
 'bert-imdb-sentiment/added_tokens.json')

In [11]:
import torch

# Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model.to(device)
model.eval()

# Sample input text
text = "The movie was absolutely fantastic and emotionally powerful."

# Tokenize
inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=256
)

# Move inputs to same device as model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Inference
with torch.no_grad():
    outputs = model(**inputs)

# Prediction
prediction = torch.argmax(outputs.logits, dim=1).item()

print("Predicted Sentiment:", "Positive" if prediction == 1 else "Negative")

Predicted Sentiment: Positive


This assignment explores both classical and modern approaches to sentiment analysis. In Problem 1, traditional NLP techniques are combined with distributed word representations using the Google News Word2Vec model. Tweets are preprocessed through normalization, lemmatization, and noise removal before being converted into dense vectors by averaging word embeddings. A multiclass Logistic Regression classifier is trained on these vectors, providing an interpretable and efficient baseline model.

Problem 2 demonstrates a deep learning-based pipeline using Hugging Face’s Transformers library. A pre-trained BERT model is fine-tuned on the IMDb dataset for binary sentiment classification. Tokenization, training, evaluation, and inference are handled using standardized APIs, ensuring scalability and reproducibility.

Challenges include high memory usage and computational cost, especially for Word2Vec loading and BERT fine-tuning. These are mitigated by batching, truncation, and limiting epochs. Together, these approaches highlight the evolution from feature-based NLP to end-to-end transformer models.